# Fine-Tuning GPT 2


## Load Sentiment model

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install transformers

In [ ]:
import numpy as np
import re
import pandas as pd
import torch
import torch.nn as nn
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import transformers
from transformers import AutoModel, BertTokenizerFast
import pickle
import random

# specify GPU
device = torch.device("cuda")

ValueError: ignored

In [ ]:
!gdown --id 1DtBqiakVmdaTRzMV9etBDOpJ0aF-TzIN

Downloading...
From: https://drive.google.com/uc?id=1DtBqiakVmdaTRzMV9etBDOpJ0aF-TzIN
To: /content/formal_train.obj
100% 5.99M/5.99M [00:00<00:00, 93.9MB/s]


In [ ]:
class CreateBatches:
  def __init__(self, path):
    self.df = pd.DataFrame({"text": pickle.load(open(path, "rb"))})
    self.df["orig_text"] = self.df["text"] 
    self.preprocess()
    self.batch_size = config["batch_size"]
  
  def preprocess(self):
    bos = gpt2_tokenizer.bos_token
    eos = "<SCI_GEN>"
    self.df["text"] = [bos + s + eos for s in self.df["text"].tolist()]
    print(self.df["text"])
    print("## Generating embeddings ##")
    self.df['tokens'] = self.df['text'].progress_apply(lambda x: gpt2_tokenizer.encode(' '+x, return_tensors="pt").to(device)[0, :config['txt_in_len']])
    print("## Decoding embeddings ##")
    self.df['query'] = self.df['tokens'].progress_apply(lambda x: gpt2_tokenizer.decode(x))
    self.df["len_tokens"] = [len(x) for x in self.df["tokens"]]
    self.df = self.df.sort_values("len_tokens")
    self.df = self.df[self.df.len_tokens > 2]
  
  def generate_padded_batches(self):
    print("## Generating padded batches ##")
    all_tokens = []
    all_attention_masks = []
    all_query_text = []
    for g, dfi in tqdm(self.df.groupby(np.arange(len(self.df)) // self.batch_size)):
        token_len = dfi.len_tokens.to_list()
        if len(set(token_len)) == 1:
          max_len = max(token_len)
          attention_mask = torch.ones(self.batch_size,max_len).to(device)
          batch_stack = torch.stack(dfi.tokens.to_list())
          query_text = dfi.orig_text
        else:
          # Padd all batch members to max batch len
          max_len = max(token_len)
          attention_mask = torch.ones(self.batch_size,max_len)
          diff =  max_len - dfi.len_tokens.to_numpy()
          for i,d in enumerate(diff):
            if d == 0: continue
            attention_mask[i, -d] = 0
          #dfi.tokens = [torch.cat([t[:-1], torch.full((1, max(token_len)- len(t)), gpt2_tokenizer.pad_token_id)[0].to(device), t[-1:]]) for t in dfi.tokens]
          batch_stack = torch.stack([torch.cat([t, torch.full((1, max(token_len)- len(t)), gpt2_tokenizer.pad_token_id)[0].to(device)]) for t in dfi.tokens])
          query_text = dfi.orig_text
        
        all_tokens.append(batch_stack)
        all_attention_masks.append(attention_mask)
        all_query_text.append(query_text.to_list())
    random.Random(2).shuffle(all_tokens)
    random.Random(2).shuffle(all_attention_masks)
    random.Random(2).shuffle(all_query_text)
    return all_tokens, all_attention_masks, all_query_text

In [ ]:
#formal_train = pickle.load(open("/content/formal_train.obj", "rb"))

#bos = "<BOS>"
#eos = "<EOS>"
#sg = "<SCI_GEN>"


#train_list = []
#train_text = ""
#for i in range(len(formal_train)):
#  train_list.append(bos + formal_train[i] + sg + formal_train[i] + eos)

#max_length = 128
#y_array = np.zeros(len(formal_train))

#train_text, temp_text, train_labels, temp_labels = train_test_split(train_list[:10000], y_array[:10000], 
#                                                                    random_state=2018, 
#                                                                    test_size=0.3, 
#                                                                    stratify=y_array[:10000])

# import BERT-base pretrained model
#bert = AutoModel.from_pretrained('bert-base-uncased')

# Load the BERT tokenizer
#tokenizer_contextual = BertTokenizerFast.from_pretrained('bert-base-uncased')
#tokenizer_contextual.add_tokens("[START]", special_tokens=True)
#tokenizer_contextual.add_tokens("[END]", special_tokens=True)

#bert.resize_token_embeddings(len(tokenizer_contextual))

#contextual_tokens_train = tokenizer_contextual.batch_encode_plus(
#    train_text,
#    max_length = max_length,
#    pad_to_max_length=True,
#    truncation=True
##)

#contextual_train_seq = torch.tensor(contextual_tokens_train['input_ids']).to(torch.device("cpu"))
#contextual_train_mask = torch.tensor(contextual_tokens_train['attention_mask']).to(torch.device("cpu"))

In [ ]:
"""
open_file = open('/content/drive/MyDrive/test/tokenized.pkl', "rb")
tokenized_paragraphs = pickle.load(open_file)

open_file2 = open('/content/drive/MyDrive/test/y.pkl','rb')
y_data = pickle.load(open_file2)

max_length = 128

numpy_array = []
counter = 0
y_array = []
for i in range(len(tokenized_paragraphs)):
      len_para = len(tokenized_paragraphs[i])
      if len_para < max_length:
        string = " ".join(tokenized_paragraphs[i])

        # remove paragraphs 
        index1 = string.find('[START]')
        index2 = string.find('[END]')
        string = string[index1+8:index2]
        print(string)

        numpy_array.append(string)
        y_array.append(y_data[i])
      else:
        counter +=1
      
numpy_array = np.asarray(numpy_array)



train_text, temp_text, train_labels, temp_labels = train_test_split(numpy_array[:5000], y_array[:5000], 
                                                                    random_state=2018, 
                                                                    test_size=0.3, 
                                                                    stratify=y_array[:5000])


val_text, test_text, val_labels, test_labels = train_test_split(temp_text, temp_labels, 
                                                                random_state=2018, 
                                                                test_size=0.5, 
                                                                stratify=temp_labels)


# import BERT-base pretrained model
bert = AutoModel.from_pretrained('bert-base-uncased')

# Load the BERT tokenizer
tokenizer_contextual = BertTokenizerFast.from_pretrained('bert-base-uncased')
tokenizer_contextual.add_tokens("[START]", special_tokens=True)
tokenizer_contextual.add_tokens("[END]", special_tokens=True)

bert.resize_token_embeddings(len(tokenizer_contextual))



contextual_tokens_test = tokenizer_contextual.batch_encode_plus(
    test_text.tolist(),
    max_length = max_length,
    pad_to_max_length=True,
    truncation=True
)

contextual_tokens_train = tokenizer_contextual.batch_encode_plus(
    train_text.tolist(),
    max_length = max_length,
    pad_to_max_length=True,
    truncation=True
)

contextual_train_seq = torch.tensor(contextual_tokens_train['input_ids']).to(torch.device("cpu"))
contextual_train_mask = torch.tensor(contextual_tokens_train['attention_mask']).to(torch.device("cpu"))

contextual_test_seq = torch.tensor(contextual_tokens_test['input_ids']).to(torch.device("cpu"))
contextual_test_mask = torch.tensor(contextual_tokens_test['attention_mask']).to(torch.device("cpu"))

#test_y = torch.tensor(test_labels)
r = torch.cuda.memory_reserved(0)
a = torch.cuda.memory_allocated(0)
f = r-a  # free inside reserved
print(f)



del temp_text
del train_labels
del temp_labels
del val_text
del test_text
del val_labels
del test_labels
"""

'\nopen_file = open(\'/content/drive/MyDrive/test/tokenized.pkl\', "rb")\ntokenized_paragraphs = pickle.load(open_file)\n\nopen_file2 = open(\'/content/drive/MyDrive/test/y.pkl\',\'rb\')\ny_data = pickle.load(open_file2)\n\nmax_length = 128\n\nnumpy_array = []\ncounter = 0\ny_array = []\nfor i in range(len(tokenized_paragraphs)):\n      len_para = len(tokenized_paragraphs[i])\n      if len_para < max_length:\n        string = " ".join(tokenized_paragraphs[i])\n\n        # remove paragraphs \n        index1 = string.find(\'[START]\')\n        index2 = string.find(\'[END]\')\n        string = string[index1+8:index2]\n        print(string)\n\n        numpy_array.append(string)\n        y_array.append(y_data[i])\n      else:\n        counter +=1\n      \nnumpy_array = np.asarray(numpy_array)\n\n\n\ntrain_text, temp_text, train_labels, temp_labels = train_test_split(numpy_array[:5000], y_array[:5000], \n                                                                    random_state=2018, \

In [ ]:
class BERT_Arch(nn.Module):

    def __init__(self, bert):
      
      super(BERT_Arch, self).__init__()

      self.bert = bert 
      
      # dropout layer
      self.dropout = nn.Dropout(0.2)
      
      # relu activation function
      self.relu =  nn.ReLU()

      # dense layer 1
      self.fc1 = nn.Linear(768,512)
      
      # dense layer 2 (Output layer)
      self.fc2 = nn.Linear(512,2)

      #softmax activation function
      self.softmax = nn.LogSoftmax(dim=1)

    #define the forward pass
    def forward(self, sent_id, mask):

      #pass the inputs to the model  
      _, cls_hs = self.bert(sent_id, attention_mask=mask, return_dict=False)
      
      x = self.fc1(cls_hs)

      x = self.relu(x)

      x = self.dropout(x)

      # output layer
      x = self.fc2(x)
      
      # apply softmax activation
      x = self.softmax(x)

      return x


# pass the pre-trained BERT to our define architecture
contextual_model = BERT_Arch(bert)


# gpu
if torch.cuda.is_available():
  device = torch.device("cuda")
  print("NICE")
else:
  device = torch.device("cpu")
#device = torch.device("cpu")

NameError: ignored

In [ ]:
%load_ext autoreload
%autoreload 2
!pip install wandb
!pip install datasets
!pip install trl
!pip install GPUtil

In [ ]:
import os, psutil
process = psutil.Process(os.getpid())
map_location=torch.device('cpu')
print(f'Before: {process.memory_info().rss}')  # in bytes 

path = '/content/drive/MyDrive/test/saved_weights.pt'
contextual_model.load_state_dict(torch.load(path, map_location ='cpu'))

_ = contextual_model.to('cpu')

print(f'After: {process.memory_info().rss}')  # in bytes 

r = torch.cuda.memory_reserved(0)
a = torch.cuda.memory_allocated(0)
f = r-a  # free inside reserved
print(f)

Before: 1265786880


NameError: ignored

# Train Model


In [ ]:
%load_ext autoreload
%autoreload 2
!pip install wandb
!pip install datasets
!pip install trl
!pip install GPUtil

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
import torch
import wandb
import time
import os
from tqdm.notebook import tqdm
import numpy as np
import pandas as pd
tqdm.pandas()

from datasets import load_dataset

from transformers import GPT2Tokenizer
from transformers import AutoModelForSequenceClassification, AutoTokenizer

from trl.gpt2 import GPT2HeadWithValueModel, respond_to_batch, GPT2Model
from trl.ppo import PPOTrainer
from trl.core import build_bert_batch_from_txt

In [ ]:
max_length = 25


config = {
    "lm_name": "gpt2",
    "tk_name": "gpt2",
    "ref_lm_name": "gpt2",
    "steps": 25600,
    "batch_size": 4,
    "forward_batch_size": 2,
    "ppo_epochs": 4,   
    "txt_in_len": max_length,
    "txt_out_len": max_length,
    "lr": 1.41e-5,
    "init_kl_coef":0.2,
    "target": 6,
    "horizon":10000,
    "gamma":1,
    "lam":0.95,
    "cliprange": .2,
    "cliprange_value":.2,
    "vf_coef":.1, 
}

In [ ]:
!gdown --i 1-P87WL_qCuSwIQZLwXb8hPkqCJsbyEej
!gdown --i 1-_eLDKeEs7Es7elJ4e2y7K32DbYjCRlZ
!gdown --i 1-iZkYQkQADPswhvWJFRra9I9DjVQBbEV
!gdown --i 1-n9ocF91o89T6Ta-hyHKLPrj0fH41vCE
!gdown --i 1-tVTAKJViIEYtMvK-Zhg27F6IsbfduPt
!gdown --i 1-xxgGLStBLi89qeEQtDSrgouZF6Puhz2
!gdown --i 1-yPbaCGCT30R80GwPpw6Q8UAmoj5GYpD
!gdown --i 1-zFviEfjztd04PCRiZR8UkJdlf76mT_Q

Downloading...
From: https://drive.google.com/uc?id=1-P87WL_qCuSwIQZLwXb8hPkqCJsbyEej
To: /content/config.json
100% 719/719 [00:00<00:00, 1.10MB/s]
Downloading...
From: https://drive.google.com/uc?id=1-_eLDKeEs7Es7elJ4e2y7K32DbYjCRlZ
To: /content/pytorch_model.bin
100% 1.44G/1.44G [00:07<00:00, 204MB/s]
Downloading...
From: https://drive.google.com/uc?id=1-iZkYQkQADPswhvWJFRra9I9DjVQBbEV
To: /content/merges.txt
100% 456k/456k [00:00<00:00, 65.8MB/s]
Downloading...
From: https://drive.google.com/uc?id=1-n9ocF91o89T6Ta-hyHKLPrj0fH41vCE
To: /content/vocab.json
100% 899k/899k [00:00<00:00, 59.4MB/s]
Downloading...
From: https://drive.google.com/uc?id=1-tVTAKJViIEYtMvK-Zhg27F6IsbfduPt
To: /content/added_tokens.json
100% 68.0/68.0 [00:00<00:00, 54.8kB/s]
Downloading...
From: https://drive.google.com/uc?id=1-xxgGLStBLi89qeEQtDSrgouZF6Puhz2
To: /content/special_tokens_map.json
100% 140/140 [00:00<00:00, 111kB/s]
Downloading...
From: https://drive.google.com/uc?id=1-yPbaCGCT30R80GwPpw6Q8UAmoj5G

In [ ]:
checkpoint = "/content/"
tokenizer_path = "/content/"

In [ ]:
wandb.init(name='r42', project='gpt2-finetune-reward', config=config, )

<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [ ]:
model_gpt = GPT2HeadWithValueModel.from_pretrained(checkpoint)
gpt2_model_ref = GPT2HeadWithValueModel.from_pretrained(checkpoint)
gpt2_tokenizer = GPT2Tokenizer.from_pretrained(tokenizer_path, bos_token = '<BOS>', eos_token = '<EOS>')
#gpt2_tokenizer.add_tokens(new_tokens=['<SCI_GEN>'], special_tokens=True)

#model_gpt = GPT2HeadWithValueModel.from_pretrained('gpt2')
#gpt2_model_ref = GPT2HeadWithValueModel.from_pretrained('gpt2')
#gpt2_tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
r = torch.cuda.memory_reserved(0)
a = torch.cuda.memory_allocated(0)
f = r-a  # free inside reserved
print(f)

Some weights of GPT2HeadWithValueModel were not initialized from the model checkpoint at /content/ and are newly initialized: ['v_head.summary.weight', 'v_head.summary.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of GPT2HeadWithValueModel were not initialized from the model checkpoint at /content/ and are newly initialized: ['v_head.summary.weight', 'v_head.summary.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


0


In [ ]:
#special_tokens = {'pad_token': '[PAD]', 'sep_token': '[SEP]'}

#gpt2_tokenizer.add_special_tokens(special_tokens)

#custom_special_tokens = {'start_token': '[START]', 'end_token': '[END]'}
#gpt2_tokenizer.add_tokens(new_tokens=custom_special_tokens, special_tokens=True)

#model_gpt.resize_token_embeddings(len(gpt2_tokenizer))
#gpt2_model_ref.resize_token_embeddings(len(gpt2_tokenizer))

In [ ]:
# create DF from  
#import pandas as pd

#print(train_text.shape)
##print(contextual_train_seq.shape)
#print(contextual_train_mask.shape)

#df = pd.DataFrame({'text': train_text, 'contextual_seq': contextual_train_seq, 'contextual_mask': contextual_train_mask})
#df['gpt2_tokens'] = df['text'].progress_apply(lambda x: gpt2_tokenizer.encode(x, return_tensors="pt").to('cuda')[0, :config['txt_in_len']])

In [ ]:
#print(response)

### Daniels Style Classifier

In [ ]:
!gdown --id 12280LmO6gFiLncXj26N4PZ6_4qhDl_WL
!unzip GYAFC_SciBERT_bs128.zip
from torch import nn
from transformers import AutoTokenizer,AutoModel, BertForSequenceClassification

style_tokenizer = AutoTokenizer.from_pretrained('allenai/scibert_scivocab_uncased')
style_model = BertForSequenceClassification.from_pretrained("/content/GYAFC_SciBERT_bs128")
_ = style_model.to(device)

softmax = nn.Softmax(dim=1)

def forward_bert(query_text, response_text, tokenizer, model):
  combined_text = query_text + response_text
  model_inputs, attention_masks = build_bert_batch_from_txt(combined_text, tokenizer, device)
  rewards = []
  for i in range(int(config['batch_size']*2/fbs)):
    res = model.forward(model_inputs[i*fbs:(i+1)*fbs],attention_masks[i*fbs:(i+1)*fbs])[0].detach()
    res = softmax(res)[:, 0]
    rewards.append(res)
  rewards = torch.cat(rewards)
  query_softmax, response_softmax = torch.split(rewards, len(response_text))
  rewards = response_softmax - query_softmax
  return rewards

Downloading...
From: https://drive.google.com/uc?id=12280LmO6gFiLncXj26N4PZ6_4qhDl_WL
To: /content/GYAFC_SciBERT_bs128.zip
100% 408M/408M [00:02<00:00, 193MB/s]
Archive:  GYAFC_SciBERT_bs128.zip
   creating: GYAFC_SciBERT_bs128/
  inflating: GYAFC_SciBERT_bs128/config.json  
  inflating: GYAFC_SciBERT_bs128/pytorch_model.bin  


Downloading:   0%|          | 0.00/385 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/228k [00:00<?, ?B/s]

### Dhruv's Content Preservation score

In [ ]:
!gdown --id 1-XPi_e6Yd29wpx9mOCrTvesPTJ9ubgZZ #requirements.txt
!gdown --id 13rDhmuFjz3d1NWYm42n7I6pQsyieaazT #sent_score_pair.py
!gdown --id 10GFDrB_C21PA1H7Hjq1SGmyjcD0QehuC #models.py
!gdown --id 10BQYSlGu1CLjFUzFMtwuxgas_Qidjl6L #pairing.py
!gdown --id 13_wkK7rsVgRlYmeoRYdxIT3-E_OJ0IOQ #utils.py
!gdown --id 10ZDlZYoF6XCV8LIscnUVnfg2ZM9tTr-m #eval_sent.py
!gdown --id 1Vss9koo31QGnBr3BYbybe7hC9Ldk06s8 #sent pair txt file
!gdown --id 19uy4-kXXDNYkvP2-Nj44imIosLKFNseE #sentence piece model
!gdown --id 19sHUsvCM_gnHEC27q7ByC-RnV2V7UfL1 # BERT model

Downloading...
From: https://drive.google.com/uc?id=1-XPi_e6Yd29wpx9mOCrTvesPTJ9ubgZZ
To: /content/requirements.txt
100% 68.0/68.0 [00:00<00:00, 215kB/s]
Downloading...
From: https://drive.google.com/uc?id=13rDhmuFjz3d1NWYm42n7I6pQsyieaazT
To: /content/score_sentence_pairs.py
100% 2.81k/2.81k [00:00<00:00, 4.59MB/s]
Downloading...
From: https://drive.google.com/uc?id=10GFDrB_C21PA1H7Hjq1SGmyjcD0QehuC
To: /content/models.py
100% 11.3k/11.3k [00:00<00:00, 17.1MB/s]
Downloading...
From: https://drive.google.com/uc?id=10BQYSlGu1CLjFUzFMtwuxgas_Qidjl6L
To: /content/pairing.py
100% 6.09k/6.09k [00:00<00:00, 9.45MB/s]
Downloading...
From: https://drive.google.com/uc?id=13_wkK7rsVgRlYmeoRYdxIT3-E_OJ0IOQ
To: /content/utils.py
100% 5.65k/5.65k [00:00<00:00, 8.66MB/s]
Downloading...
From: https://drive.google.com/uc?id=10ZDlZYoF6XCV8LIscnUVnfg2ZM9tTr-m
To: /content/evaluate_sts.py
100% 11.4k/11.4k [00:00<00:00, 18.1MB/s]
Downloading...
From: https://drive.google.com/uc?id=1Vss9koo31QGnBr3BYbybe7h

In [ ]:
!pip install -r /content/requirements.txt

     |████████████████████████████████| 1.2 MB 5.1 MB/s 


In [ ]:
import argparse
from models import load_model

parser = argparse.ArgumentParser()
parser.add_argument("--load-file", default = "/content/model.para.lc.100.pt",help="path to saved model")
parser.add_argument("--sp-model", default = "/content/paranmt.model",help="sentencepiece model to use")
parser.add_argument("--gpu", default=1, type=int, help="whether to train on gpu")

args = parser.parse_args(args=[])

content_model, _ = load_model(None, args)
content_model.eval()

Averaging(
  (sim_loss): MarginRankingLoss()
  (cosine): CosineSimilarity()
  (embedding): Embedding(82983, 1024)
)


Averaging(
  (sim_loss): MarginRankingLoss()
  (cosine): CosineSimilarity()
  (embedding): Embedding(82983, 1024)
)

In [ ]:
import argparse
import numpy as np
from sacremoses import MosesTokenizer
from models import load_model
from sacremoses import MosesTokenizer
from utils import Example

def cosine(u, v):
    return np.dot(u, v) / (np.linalg.norm(u) * np.linalg.norm(v))

class FileSim:

    def __init__(self):
        self.similarity = lambda s1, s2: np.nan_to_num(cosine(np.nan_to_num(s1), np.nan_to_num(s2)))

    def score(self, params, batcher, s1, s2):
        input1 = [s1]
        input2 = [s2]
        sys_scores = []
        for ii in range(0, len(input1), params.batch_size):
            batch1 = input1[ii:ii + params.batch_size]
            batch2 = input2[ii:ii + params.batch_size]

            # we assume get_batch already throws out the faulty ones
            if len(batch1) == len(batch2) and len(batch1) > 0:
                enc1 = batcher(params, batch1)
                enc2 = batcher(params, batch2)

                for kk in range(enc2.shape[0]):
                    sys_score = self.similarity(enc1[kk], enc2[kk])
                    sys_scores.append(sys_score)

        return sys_scores

def batcher(params, batch):
    new_batch = []
    for p in batch:
        if params.tokenize:
            tok = params.entok.tokenize(p, escape=False)
            p = " ".join(tok)
        if params.lower_case:
            p = p.lower()
        p = params.sp.EncodeAsPieces(p)
        p = " ".join(p)
        p = Example(p, params.lower_case)
        p.populate_embeddings(params.model.vocab, params.model.zero_unk, params.model.ngrams)
        new_batch.append(p)
    x, l = params.model.torchify_batch(new_batch)
    vecs = params.model.encode(x, l)
    return vecs.detach().cpu().numpy()

In [ ]:
def evaluate(args, model, sentences_list1, sentences_list2):

    entok = MosesTokenizer(lang='en')

    from argparse import Namespace

    new_args = Namespace(batch_size=32, entok=entok, sp=model.sp,
                     params=args, model=model, lower_case=model.args.lower_case,
                     tokenize=model.args.tokenize)
  
    s = FileSim()
    scores = []
    for i in range(len(sentences_list1)):
        score = s.score(new_args, batcher, sentences_list1[i], sentences_list2[i])
        scores.append(score)
    return torch.tensor(torch.transpose(torch.tensor(scores), 1, 0)[0])

In [ ]:
print(evaluate(args, content_model, "My cat.", "Stray cat."))

tensor([ 5.3805e-01,  8.7628e-02, -1.3197e-02,  3.5579e-04,  6.2122e-02,
         4.5637e-02, -1.3192e-01])


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  app.launch_new_instance()


### Continue Dhruv code

In [ ]:
import pickle

In [ ]:
path = "/content/formal_train.obj"
cb = CreateBatches(path)
all_tokens, all_attention_masks, all_query_text = cb.generate_padded_batches()

NameError: ignored

In [ ]:
import gc
import GPUtil
import torch

#del query_tensors
_ = model_gpt.to("cuda")
_ = gpt2_model_ref.to("cuda")
gc.collect()
#torch.cuda.empty_cache()
GPUtil.showUtilization(all=True)

| ID | Name                 | Serial        | UUID                                     || GPU temp. | GPU util. | Memory util. || Memory total | Memory used | Memory free || Display mode | Display active |
-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|  0 | Tesla P100-PCIE-16GB | 0320617039077 | GPU-ef404b98-9c1c-efc6-7c7c-9ad331289fb9 ||       39C |        0% |          33% ||      16280MB |      5399MB |     10881MB || Enabled      | Disabled       |


In [ ]:
ppo_trainer = PPOTrainer(model_gpt, gpt2_model_ref, **config)
fbs = config['forward_batch_size']
import gc
#GPUtil.showUtilization(all=True)
#CUDA_LAUNCH_BLOCKING = "1"

#for epoch in tqdm(range(int(np.ceil(config["steps"]/config['batch_size'])))):
for epoch in tqdm(range(int(len(all_tokens)))):
    torch.cuda.empty_cache()
    logs = dict()
    game_data = dict()
    timing = dict()
    t0 = time.time()
    gc.collect()

    game_data['query'] = all_query_text[epoch]
    query_tensors = all_tokens[epoch]
    max_len = query_tensors.size()[1]

    #### get response from gpt2
    t = time.time()

    response_tensors = []
    
    #print(f'max len {max_len}')
    #print(query_tensors[0])
  
    for i in tqdm(range(int(len(all_tokens[epoch])/fbs))):
        response  = respond_to_batch(model_gpt, query_tensors[i*fbs:(i+1)*fbs],
                                     txt_len=max_len, top_k =50,top_p =0.7)
        response_tensors.append(response)
      
    #print(response_tensors)
    response_tensors = torch.cat(response_tensors)
    
    game_data['response'] = [gpt2_tokenizer.decode(response_tensors[i, :])  for i in range(len(all_tokens[epoch]))]

    #print(game_data['query'])
    #print(game_data['response'])

    timing['time/get_response'] = time.time()-t

    #### tokenize text for contextual model
    t = time.time()
    
    rewards = []


    timing['time/get_sentiment_preds'] = time.time()-t


    #print(query_tensors.shape)
    #print(response_tensors.shape)

    # Style Classifier Reward #
    #rewards_style = forward_bert(game_data["query"],game_data["response"],style_tokenizer, style_model)

    # Content preservation
    rewards_content = evaluate(args, content_model ,game_data["query"],game_data["response"]).to('cuda')

    #rewards = torch.sum(torch.stack([rewards_style, rewards_content]), dim=0)
    rewards = rewards_content

    ###########################
  
    #### Run PPO training 
    t = time.time()
    stats = ppo_trainer.step(query_tensors.to('cuda'), response_tensors.to('cuda'), rewards.to('cuda'))
    timing['time/optimization'] = time.time()-t
      
    #### Log everything
    timing['time/epoch'] = time.time()-t0
    table_rows = [list(r) for r in zip(game_data['query'], game_data['response'], rewards.cpu().tolist())]
    logs.update({'game_log':wandb.Table(
        columns=['query', 'response', 'reward'],
        rows=table_rows)})
    logs.update(timing)
    logs.update(stats)
    logs['env/reward_mean'] = torch.mean(rewards).cpu().numpy()
    logs['env/reward_std'] = torch.std(rewards).cpu().numpy()
    logs['env/reward_dist'] = rewards.cpu().numpy()

    if epoch % 50 == 0: 
      print(f"Saving model at epoch {epoch}")
      model_gpt.save_pretrained('/content/drive/MyDrive/test/gpt2/')
      gpt2_tokenizer.save_pretrained('/content/drive/MyDrive/test/gpt2Tokenizer/')

    wandb.log(logs)

  0%|          | 0/26141 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  app.launch_new_instance()


Saving model at epoch 0


  0%|          | 0/2 [00:00<?, ?it/s]

...

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]